<a href="https://colab.research.google.com/github/lanyu0322/phd_semester1/blob/master/wamp_slope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import time
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter as uf
from scipy.optimize import least_squares

#plt.style.use("bmh")
plt.rcParams["figure.figsize"] = [7, 5]
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Define 4 pop model

In [0]:
# -- read in the average weekend counts
fname = os.path.join("drive", "My Drive", "lwir", "data", "nycdot", 
                     "avg_weekend_ts.feather")
weekend = uf(pd.read_feather(fname)["avg_counts"].values, 8)

In [0]:
# -- read temp and humidity data
tname = os.path.join("drive", "My Drive", "lwir", "data", "4_pop_fit", 
                     "temp_humidity.csv")
temp = pd.read_csv(tname)

In [0]:
# -- define gaussian
def gauss(xarr, x0, sig, scl):
  """ Create a 1D Gaussian """
  
  amp = scl / (sig * np.sqrt(2.0 * np.pi))
  val = -0.5 * ((xarr - x0) / sig)**2
  
  return amp * np.exp(val)


# -- define the model
def pop_model(param, xval):

  # -- determine which model is being used from length of param array
  nparam = len(param)

  # -- 3 population model
  if nparam == 10:
    m1, m2, m3, sd1, sd2, sd3, scl1, scl2, scl3, off = param
    wamp = 0.0

  # -- 4 population model
  elif nparam == 11:
    m1, m2, m3, sd1, sd2, sd3, scl1, scl2, scl3, wamp, off = param

  # -- ill-defined param array
  else:
    print("Nparam must be 10 (3pop) or 11 (4pop) model!")
    return None

  model = gauss(xval, m1, sd1, scl1) + gauss(xval, m2, sd2, scl2) + \
    gauss(xval, m3, sd3, scl3) + wamp * wendvals + off

  return model


# -- define model error
def res(param, xval, yval):

  return yval - pop_model(param, xval)

In [0]:
# -- set inital guess
avgs  = [35, 47, 70] 
sigs  = [3, 3, 3]
scls  = [5, 5, 5]
off   = [0.5]
wamp = [6]
param = avgs + sigs + scls + wamp + off


In [0]:
# -- set fit and model parameters and 
wendvals = weekend
xval_final = np.arange(96)
bounds_lo = [24., 44., 64., 2.0, 2.0, 2.0, 0, 0, 0, 0, -np.inf]
bounds_hi = [44., 64., 80., 8., 8., 8., np.inf, np.inf, np.inf, np.inf, np.inf]

# Get time series data for each camera each day

In [0]:
# -- read the outliers have no three peak
bind = [175, 409, 172, 176, 165, 170, 173, 360, 501, 537, 545, 795, 547, 845, 
        845, 899, 891, 934, 940, 970, 475, 989, 791, 988, 291, 530, 909, 248, 
        500, 177, 968, 517, 805, 932, 432, 452, 847, 790, 969, 841, 446, 893, 
        715]

In [0]:
# -- read multi_files and combine them together
path = r'drive/My Drive/lwir/data/nycdot/counts_wd_cam'
all_files = glob.glob(path + "/*.feather")

li = []
cnum = []

for filename in all_files:
  valid = True
  for b_id in bind:
    if (str(b_id) in filename):
      valid = False
      break
  if valid:
    df = pd.read_feather(filename)
    cum = filename.split("_")[-1].split("_")[0]
    li.append(df)
    cnum.append(cum)

frame = pd.concat(li, axis=0, ignore_index=True)

In [0]:
# -- create day and time column
frame["day"] = frame["date"].dt.date
frame["time"] = frame["date"].dt.time

In [0]:
# -- get available date data
day = frame.groupby("day").count().reset_index()

In [0]:
# -- groupby day and get the values
ff1a = frame.groupby("time")

avg_frame = []
for ind, gp in ff1a:
  avg_frame.append(gp["count"].values)

In [0]:
# -- replace nans to nearest average counts
avg_frame = pd.DataFrame(avg_frame)
fnan = avg_frame.ffill()
bnan = avg_frame.bfill()
avg_count = 0.5*(fnan + bnan).T

In [0]:
# -- add date for groupby
def new_sratio(sratio):
  sratio_day = []
  for index, row in sratio.iterrows():
   
    day_index = index % 31
    new_row = []
    new_row.append(row["sratio"])
    new_row.append(day["day"].iloc[day_index]) #add a new sublist to each sratio
    sratio_day.append(new_row)

  return sratio_day

# 1a. For a given day, calculate the sratio for each camera and average.

In [0]:
# -- transfer to dataframe 
avg_la = pd.DataFrame(avg_count)


cam_avg = []
for i in range(115):
  if i == 0:
    avg_cam = avg_la.iloc[0:31].mean()
  else:
    ii = i * 31
    jj = ii + 31
    avg_cam = avg_la.iloc[ii:jj].mean()
  cam_avg.append(avg_cam)
  
cam_avg = pd.DataFrame(cam_avg)

In [0]:
# -- set fit and model parameters and 
wendvals = weekend
xval_final = np.arange(96)
bounds_lo = [24., 44., 64., 2.0, 2.0, 2.0, 0, 0, 0, 0, -np.inf]
bounds_hi = [44., 64., 80., 8., 8., 8., np.inf, np.inf, np.inf, np.inf, np.inf]
# -- optimize
plsq = []
for ii in range(len(cam_avg)):
  #print("working on day {0}".format(ii))

  plsq.append(least_squares(res, param, bounds=(bounds_lo, bounds_hi), 
                            args=(xval_final, cam_avg.iloc[ii])))

In [0]:
# -- put fit results into DataFrame
result_1a = pd.DataFrame(plsq)["x"].values

df1a = pd.DataFrame(np.vstack(result_1a), 
                     columns=["m1", "m2", "m3", "sd1", "sd2", "sd3", 
                              "scl1", "scl2", "scl3", "wamp", "off"])

# Get the dropped camera ID

In [0]:
# -- add camera number
df1a["cnum"] = cnum
import re
import statsmodels.formula.api as sm

In [0]:
# -- read in the weekday counts
camera_fname = os.path.join("drive", "My Drive", "lwir", "data", "nycdot", 
                            "cams_ft_wd.feather")
camera = pd.read_feather(camera_fname)

# -- read cam id for outliers that have no three peak
bind = [175, 409, 172, 176, 165, 170, 173, 360, 501, 537, 545, 795, 547, 845, 
        845, 899, 891, 934, 940, 970, 475, 989, 791, 988, 291, 530, 909, 248, 
        500, 177, 968, 517, 805, 932, 432, 452, 847, 790, 969, 841, 446, 893, 
        715]

In [0]:
# -- define add date

def add_date(count):
  count_day = []
  for index, row in count.iterrows():
   
    day_index = index % 31
    new_row = []
    for i in range(len(row)):
      new_row.append(row[i])
    new_row.append(day["day"].iloc[day_index]) #add a new sublist to each row
    count_day.append(new_row)

  return count_day

In [0]:
# # coefs = []
# # wcuts = np.linspace(0.00275, 0.0040, 50)
# # for wcut in wcuts:
# # # -- select cams with wamp < wcut

#   drop_camera = df1a[df1a["wamp"] > 0.00275]
#   drop_cam = drop_camera[drop_camera["wamp"] > 0.0040]
#   dcnum = drop_cam["cnum"]
#   # -- keep only numeric
#   drop_id = dcnum.str.replace(r'[^0-9]+', '')
#   id = []
#   for e in drop_id:
#     res = e[1:]
#     id.append(res)
#   id = pd.DataFrame(id)
#   id["id"] = id
#   # -- get the lat/lon of the dropped cameras
#   camera["cam_id"] = camera["cam_id"].astype(str)
#   cam_id = camera.merge(id, left_on="cam_id", right_on="id")
#   dcnum = pd.DataFrame(dcnum)
#   bad_camera = []
#   for v in dcnum.values.tolist():
#     bad_camera.append(int(v.pop().split(".")[0]))
#     final_bad_cam = list(set(bind + bad_camera))
#   # -- read multi_files and combine them together
#   path = r'drive/My Drive/lwir/data/nycdot/counts_wd_cam'
#   all_files = glob.glob(path + "/*.feather")

#   cam = []
#   num = []

#   for filename in all_files:
#     valid = True
#     for b_id in final_bad_cam:
#       if (str(b_id) in filename):
#         valid = False
#         break
#     if valid:
#       df0 = pd.read_feather(filename)
#       cum = filename.split("_")[-1].split("_")[0]
#       cam.append(df0)
#       num.append(cum)

#   frame_drop = pd.concat(cam, axis=0, ignore_index=True)
#   # -- create day and time column
#   frame_drop["day"] = frame_drop["date"].dt.date
#   frame_drop["time"] = frame_drop["date"].dt.time
#   # -- get available date data
#   day_drop = frame_drop.groupby("day").count().reset_index()  
#   # -- groupby day and get the values

#   fft = frame_drop.groupby("time")

#   avg_frame_drop = []
#   for ind, gp in fft:
#     avg_frame_drop.append(gp["count"].values)
#   # -- replace nans to nearest average counts
#   avg_frame_drop = pd.DataFrame(avg_frame_drop)
#   fnan_drop = avg_frame_drop.ffill()
#   bnan_drop = avg_frame_drop.bfill()
#   avg_count_drop = 0.5*(fnan_drop + bnan_drop).T
#   # -- get the counts
#   avg_2b = avg_count_drop.values.copy()
#   # -- standardized data
#   avg_2b_m = avg_2b.mean(axis=1, keepdims=True)
#   avg_2b_std = avg_2b.std(axis=1, keepdims=True)
#   avg_2b_st = (avg_2b - avg_2b_m) / (avg_2b_std + (avg_2b_std == 0))
#   avg_2b_st = pd.DataFrame(avg_2b_st)
#   # -- get available date data
#   day = frame.groupby("day").count().reset_index()
#   # -- add date for st counts
#   avg_2b_date = pd.DataFrame(add_date(avg_2b_st))
#   # -- rename dataframe 96 to day
#   avg_2b_date = avg_2b_date.rename(columns = {96:'day'})
#   # -- get avg_st counts
#   avg_std_date = avg_2b_date.groupby("day").mean()

#   # -- optimize
#   wendvals = weekend
#   xval_final = np.arange(96)
#   bounds_lo = [24., 44., 64., 2.0, 2.0, 2.0, 0, 0, 0, 0, -np.inf]
#   bounds_hi = [44., 64., 80., 8., 8., 8., np.inf, np.inf, np.inf, np.inf, np.inf]

#   plsq_2b = []
#   for ii in range(len(avg_std_date)):
#     plsq_2b.append(least_squares(res, param, bounds=(bounds_lo, bounds_hi), 
#                               args=(xval_final, avg_std_date.iloc[ii])))
    
# #   # -- put fit results into DataFrame
# #   result_2b = pd.DataFrame(plsq_2b)["x"].values
# #   df2b = pd.DataFrame(np.vstack(result_2b), 
# #                       columns=["m1", "m2", "m3", "sd1", "sd2", "sd3", 
# #                               "scl1", "scl2", "scl3", "wamp", "off"])
# #   # -- calculate sratio 
# #   df2b["sratio"] = df2b["scl2"] / (0.5 * (df2b["scl1"] + df2b["scl3"]))
# #   # -- add date column
# #   sratio_2b = pd.DataFrame(new_sratio(df2b))
# #   sratio_2b["wamp"] = df2b["wamp"]
# #   sratio_2b["scl1"] = df2b["scl1"]
# #   sratio_2b["scl2"] = df2b["scl2"]
# #   sratio_2b["scl3"] = df2b["scl3"]
# #   sratio_2b.columns = ["sratio", "date", "wamp", "scl1", "scl2", "scl3"]
# #   # -- subset humidity and temp to our research period
# #   sratio_2b["date"] = sratio_2b["date"].astype(str)
# #   temp_day_2b = sratio_2b.merge(temp, left_on="date", right_on="day")
# #   # -- first build the model
# #   model = sm.ols("sratio ~ temp", data=temp_day_2b)
# #   # -- now fit the model to the data
# #   result = model.fit()
# #   coefs.append(result.coef_[0])

# # plot(wcuts, coefs)

# 从这里开始

In [0]:
drop_camera = df1a[df1a["wamp"] > 0.00275]
drop_cam = drop_camera[drop_camera["wamp"] > 0.0040]
dcnum = drop_cam["cnum"]
  # -- keep only numeric
drop_id = dcnum.str.replace(r'[^0-9]+', '')
id = []
for e in drop_id:
  res = e[1:]
  id.append(res)
id = pd.DataFrame(id)
id["id"] = id
  # -- get the lat/lon of the dropped cameras
camera["cam_id"] = camera["cam_id"].astype(str)
cam_id = camera.merge(id, left_on="cam_id", right_on="id")
dcnum = pd.DataFrame(dcnum)
bad_camera = []
for v in dcnum.values.tolist():
  bad_camera.append(int(v.pop().split(".")[0]))
  final_bad_cam = list(set(bind + bad_camera))
  # -- read multi_files and combine them together
path = r'drive/My Drive/lwir/data/nycdot/counts_wd_cam'
all_files = glob.glob(path + "/*.feather")

cam = []
num = []

for filename in all_files:
  valid = True
  for b_id in final_bad_cam:
    if (str(b_id) in filename):
      valid = False
      break
  if valid:
    df0 = pd.read_feather(filename)
    cum = filename.split("_")[-1].split("_")[0]
    cam.append(df0)
    num.append(cum)

frame_drop = pd.concat(cam, axis=0, ignore_index=True)
  # -- create day and time column
frame_drop["day"] = frame_drop["date"].dt.date
frame_drop["time"] = frame_drop["date"].dt.time
  # -- get available date data
day_drop = frame_drop.groupby("day").count().reset_index()  
  # -- groupby day and get the values

fft = frame_drop.groupby("time")

avg_frame_drop = []
for ind, gp in fft:
  avg_frame_drop.append(gp["count"].values)
  # -- replace nans to nearest average counts
avg_frame_drop = pd.DataFrame(avg_frame_drop)
fnan_drop = avg_frame_drop.ffill()
bnan_drop = avg_frame_drop.bfill()
avg_count_drop = 0.5*(fnan_drop + bnan_drop).T
  # -- get the counts
avg_2b = avg_count_drop.values.copy()
  # -- standardized data
avg_2b_m = avg_2b.mean(axis=1, keepdims=True)
avg_2b_std = avg_2b.std(axis=1, keepdims=True)
avg_2b_st = (avg_2b - avg_2b_m) / (avg_2b_std + (avg_2b_std == 0))
avg_2b_st = pd.DataFrame(avg_2b_st)
  # -- get available date data
day = frame.groupby("day").count().reset_index()
  # -- add date for st counts
avg_2b_date = pd.DataFrame(add_date(avg_2b_st))
  # -- rename dataframe 96 to day
avg_2b_date = avg_2b_date.rename(columns = {96:'day'})
  # -- get avg_st counts
avg_std_date = avg_2b_date.groupby("day").mean()



In [127]:
  # -- optimize
wendvals = weekend
xval_final = np.arange(96)
bounds_lo = [24., 44., 64., 2.0, 2.0, 2.0, 0, 0, 0, 0, -np.inf]
bounds_hi = [44., 64., 80., 8., 8., 8., np.inf, np.inf, np.inf, np.inf, np.inf]

plsq_2b = []
for ii in range(len(avg_std_date)):
  print("working on day {0}".format(ii))
  plsq_2b.append(least_squares(res, param, bounds=(bounds_lo, bounds_hi), args=(xval_final, 
                                                                                avg_std_date.iloc[ii])))

working on day 0


TypeError: ignored